In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.read_csv(r'..\Data_TFG\desk_deskdefault\tfg_counts_desktop.csv', index_col=0)
df

In [ ]:
grupos = (pd.DataFrame(df.columns.to_series()
             .str.split('_').to_dict())).iloc[0,:].to_list()
grupos

In [ ]:
unicos, cuentas = np.unique(grupos, return_counts=True)
criba = np.min(cuentas)
df_criba = df.loc[df.sum(axis=1) > criba, :].T
df_criba

In [ ]:
from scipy.stats.mstats import gmean
df_criba = df_criba.replace(to_replace=0, value=1e-9).div(df_criba.replace(to_replace=0, value=1e-9).apply(gmean, axis=1), axis=0).apply(np.log)  # tabla transformada
df_criba

In [ ]:
print(df_criba.sum(axis=1))
print('')
print(df_criba.sum(axis=1).describe())

In [ ]:
from sklearn.preprocessing import StandardScaler
escalador = StandardScaler()
df_esc = escalador.fit_transform(df_criba)
df_esc

# Elastic Net

In [ ]:
from sklearn.linear_model import LogisticRegression
mielnet = LogisticRegression(penalty='elasticnet', solver='saga')
mielnet.fit(df_esc, grupos)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
grupo_pred = mielnet.predict(df_esc)
cnf = confusion_matrix(grupos, grupo_pred)
print(f'Accuracy: {accuracy_score(grupos, grupo_pred)}')
sns.heatmap(cnf, annot=True)

In [ ]:
relevancias1 = pd.DataFrame({'OTU' : df_criba.columns, 'Relevancia' : np.abs(mielnet.coef_[0])})
relevancias1

# Lasso

In [ ]:
from sklearn.linear_model import LogisticRegression
milasso = LogisticRegression(penalty='l1', solver='liblinear')
milasso.fit(df_esc, grupos)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
grupo_pred = milasso.predict(df_esc)
cnf = confusion_matrix(grupos, grupo_pred)
print(f'Accuracy: {accuracy_score(grupos, grupo_pred)}')
sns.heatmap(cnf, annot=True)

In [ ]:
relevancias1 = pd.DataFrame({'OTU' : df_criba.columns, 'Relevancia' : np.abs(milasso.coef_[0])})
relevancias1

# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
mibosque = RandomForestClassifier()
mibosque.fit(df_esc, grupos)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
grupo_pred = mibosque.predict(df_esc)
cnf = confusion_matrix(grupos, grupo_pred)
sns.heatmap(cnf, annot=True)

In [ ]:
relevancias1 = pd.DataFrame({'OTU' : df_criba.columns, 'Relevancia' : mibosque.feature_importances_})
relevancias1